# Utilities

In [1]:
import math

In [2]:
def single_payment_compound_amount_factor(
    interest_rate : float,
    num_periods : int,
) -> float:
    assert 0 < interest_rate < 1.0
    assert num_periods >= 1
    if not isinstance(interest_rate, float): interest_rate = float(interest_rate)
    if not isinstance(num_periods, int): num_periods = int(num_periods)

    return math.pow(1.0 + interest_rate, num_periods)

def single_payment_present_worth_factor(
    interest_rate : float,
    num_periods : int,
) -> float:
    assert 0 < interest_rate < 1.0
    assert num_periods >= 1
    if not isinstance(interest_rate, float): interest_rate = float(interest_rate)
    if not isinstance(num_periods, int): num_periods = int(num_periods)

    return math.pow(1.0 + interest_rate, -num_periods)

def uniform_series_compound_amount_factor(
    interest_rate : float,
    num_periods : int,
) -> float:
    assert 0 < interest_rate < 1.0
    assert num_periods >= 1
    if not isinstance(interest_rate, float): interest_rate = float(interest_rate)
    if not isinstance(num_periods, int): num_periods = int(num_periods)

    return (math.pow(1.0 + interest_rate, num_periods) - 1.0) / interest_rate

def uniform_series_present_worth_factor(
    interest_rate : float,
    num_periods : int,
) -> float:
    assert 0 < interest_rate < 1.0
    assert num_periods >= 1
    if not isinstance(interest_rate, float): interest_rate = float(interest_rate)
    if not isinstance(num_periods, int): num_periods = int(num_periods)

    _temp = math.pow(1.0 + interest_rate, num_periods)
    return (_temp - 1.0) / (interest_rate * _temp)

def sinking_fund_factor(
    interest_rate : float,
    num_periods : int,
) -> float:
    assert 0 < interest_rate < 1.0
    assert num_periods >= 1
    if not isinstance(interest_rate, float): interest_rate = float(interest_rate)
    if not isinstance(num_periods, int): num_periods = int(num_periods)
    
    return interest_rate / (math.pow(1.0 + interest_rate, num_periods) - 1.0)

def capital_recovery_factor(
    interest_rate : float,
    num_periods : int,
) -> float:
    assert 0 < interest_rate < 1.0
    assert num_periods >= 1
    if not isinstance(interest_rate, float): interest_rate = float(interest_rate)
    if not isinstance(num_periods, int): num_periods = int(num_periods)

    return interest_rate / (1.0 - math.pow(1.0 + interest_rate, -num_periods))

# Problem

You are part of a committee tasked by a larger engineering team to come up with strategies for the next seven-year horizon.
The committee shortlisted three (presumably mutually exclusive) strategies
-- codenamed Pike, Bonaparte, and Rizal --
crafted to give residual values at the end of the said study period.
The team set an MARR of 8 % per year.
Summary economic values for the strategies are tabulated below.

|                 | Pike   | Bonaparte | Rizal  |
|-----------------|--------|-----------|--------|
| Initial costA   | 55k    | 45k       | 80k    |
| Annual expenses | 6.25k  | 8.55k     | 3.2k   |
| Annual revenue  | 18.25k | 16.75k    | 20.2k  |
| Residual value  | 18k    | 3.75k     | 22k    |
| IRR (per year)  | 15.9 % | 7.9 %     | 14.6 % |

Which alternative will you recommend to the committee?
Justify using incremental investment analysis.

In [3]:
MARR = 0.10
T = 7
monetary_unit_scaling = 1e3

# Pike, Bonaparte, Rizal
strategies = ["Pike", "Bonaparte", "Rizal"]
Cs = [55.0, 45.0, 80.0]
Ss = [18.0, 3.75, 22.0]
Es = [6.25, 8.55, 3.20]
Rs = [18.25, 16.75, 20.2]

ERs = [None, None, None]
PWs = [None, None, None]

for i, (C, S, E, R) in enumerate(zip(Cs, Ss, Es, Rs)):
    ERs[i] = math.pow(
        (S + (R * uniform_series_compound_amount_factor(MARR, T))) \
            / (C + (E * uniform_series_present_worth_factor(MARR, T))),
        (1.0 / T)
    ) - 1.0

    PWs[i] = (S * single_payment_present_worth_factor(MARR, T)) \
            + (R * uniform_series_present_worth_factor(MARR, T)) \
            - C \
            - (E * uniform_series_present_worth_factor(MARR, T))

for i, (strat, ER, PW) in enumerate(zip(strategies, ERs, PWs)):
    print(f"For an MARR of {MARR * 100} % per year, ", end="")
    print(f"{strat} has an ERR of {ER * 100 :.5} % ({'>=' if (ER >= MARR) else '<'} {MARR*100} %) ", end="")
    print(f"and a PW of {PW * monetary_unit_scaling :.5} ({'>=' if (PW >= 0) else '<'} 0).")

For an MARR of 10.0 % per year, Pike has an ERR of 12.193 % (>= 10.0 %) and a PW of 1.2658e+04 (>= 0).
For an MARR of 10.0 % per year, Bonaparte has an ERR of 9.4186 % (< 10.0 %) and a PW of -3154.6 (< 0).
For an MARR of 10.0 % per year, Rizal has an ERR of 12.177 % (>= 10.0 %) and a PW of 1.4053e+04 (>= 0).


Bonaparte is outright rejected.
The contention is now between Pike and Rizal,
and they make apparent the inconsistent ranking problem.

We choose Pike to be the base alternative,
as it has the lesser capital investment.
We now ask:
"Is Rizal's incremental capital and incremental annual expenses
compensated by its incremental annual revenues and incremental residual value?"

In [4]:
incremental_C = Cs[2] - Cs[0]
incremental_S = Ss[2] - Ss[0]
incremental_E = Es[2] - Es[0]
incremental_R = Rs[2] - Rs[0]

incremental_ER = math.pow(
    (incremental_S + (incremental_R * uniform_series_compound_amount_factor(MARR, T))) \
    / (incremental_C + (incremental_E * uniform_series_present_worth_factor(MARR, T))),
    (1.0 / T)
) - 1.0

incremental_PW = (incremental_S * single_payment_present_worth_factor(MARR, T)) \
    + (incremental_R * uniform_series_present_worth_factor(MARR, T)) \
    - incremental_C \
    - (incremental_E * uniform_series_present_worth_factor(MARR, T))

print(f"ERR for incremental cashflow: {incremental_ER * 100 :.5f} % {'>=' if (incremental_ER >= MARR) else '<'} {MARR * 100 :.5f} %")
print(f"PW for incremental cashflow: {incremental_PW * monetary_unit_scaling} {'>=' if (incremental_PW >= 0) else '<'} 0")

ERR for incremental cashflow: 12.04176 % >= 10.00000 %
PW for incremental cashflow: 1394.7265613874952 >= 0


It is thus justifiable to move to Rizal from Pike.
As such, Rizal is the preferred strategy.